<a href="https://colab.research.google.com/github/Shopping-Yuan/bert_project/blob/main/keyword_sorting/bert_keyword_sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# 安裝keras-bert函式庫
!pip -q install keras-bert
!pip -q install keras-transformer
!gdown --id "1hEPuCU0EuEh0F5MgCwFx39emivhisku5" --output multi_cased_L-12_H-768_A-12.zip
!unzip multi_cased_L-12_H-768_A-12.zip

Downloading...
From: https://drive.google.com/uc?id=1hEPuCU0EuEh0F5MgCwFx39emivhisku5
To: /content/multi_cased_L-12_H-768_A-12.zip
663MB [00:03, 191MB/s]
Archive:  multi_cased_L-12_H-768_A-12.zip
replace multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [49]:
# 讀取預訓練模型
import os
pretrained_path = 'multi_cased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
# 模型預訓練的字表
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [51]:
# BERT 透過 Subword 的方式讓辭典長度最大就是 21128
token_dict = {}
with open(vocab_path, 'r', encoding='utf8') as f:
    for line in f.readlines():
        token = line.strip()
        token_dict[token] = len(token_dict)
print("辭典長度:", len(token_dict))

辭典長度: 119547


In [52]:
#注意單字表是從1開始和Token的值會有差異
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict)

In [53]:
#載入jieba字典
import jieba.analyse
import os
from urllib.request import urlretrieve
if not os.path.exists ("dict.txt.big"):
  url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
  urlretrieve(url, "dict.txt.big")
jieba.set_dictionary ('dict.txt.big') 

In [54]:
# 連 Server 依日期分別抓取訓練集和測試集
!pip install pymysql
# !gdown --id "1Wv0l3XtyWd3DI82jx0oQpVC945WDIxd7" --output LinkMariaDB.py
import LinkMariaDB
hostName = '34.80.102.35'
portNum = 3306
DBName = 'db_blogler'
username = 'shop'
password = '12345678'
CharSet = "utf8"

DB = LinkMariaDB.OpMariaSQL(hostName, portNum, username, password, DBName, CharSet)
raw_data_train = DB.IntputFindCmd(f"SELECT content FROM main WHERE (label_1 = '1' OR label_2 = '1') AND pub_month <= 202001")
raw_data_test = DB.IntputFindCmd(f"SELECT content FROM main WHERE (label_1 = '1' OR label_2 = '1') AND pub_month > 202001")

IntputFindCmd:
 SELECT content FROM main WHERE (label_1 = '1' OR label_2 = '1') AND pub_month <= 202001
IntputFindCmd:
 SELECT content FROM main WHERE (label_1 = '1' OR label_2 = '1') AND pub_month > 202001


In [55]:
#準備載入關鍵詞列表,將關鍵詞表中的字轉換為簡體,加入jieba字典
!pip install opencc-python-reimplemented
from opencc import OpenCC
import re
cc = OpenCC('t2s')
keyword_list_convert = []
!gdown --id "1TWzZQyyVvcfZfw4PObzaCIvGZbqWNdFN" --output keyword_list.txt
with open('keyword_list.txt', 'r') as f:
  t = f.readlines()
  for i in t:
    keyword_list_convert.append(cc.convert(i))
with open("keyword_list_convert.txt", "w") as output:
  for item in keyword_list_convert:
    output.write("%s" % item)
medium_dict_path = "keyword_list_convert.txt"
if os.path.exists(medium_dict_path):
    jieba.load_userdict(medium_dict_path)
keyword_list = []
for n in keyword_list_convert:
  keyword_list.append(n.strip("\n"))
jieba.re_han_default = re.compile('(.+)', re.U)

Downloading...
From: https://drive.google.com/uc?id=1TWzZQyyVvcfZfw4PObzaCIvGZbqWNdFN
To: /content/keyword_list.txt
100% 769k/769k [00:00<00:00, 12.2MB/s]


Building prefix dict from /content/dict.txt.big ...
Loading model from cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.031 seconds.
Prefix dict has been built successfully.


In [56]:
#此函數用來且移除原文的標點符號
import numpy as np
punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')
def data(x):
  words = ",".join(filter(lambda x: x not in punct, jieba.cut(x)))
  return words

In [57]:
#將訓練集與測試集的資料轉為簡體並移除原文的標點符號
import pandas as pd
df_train = pd.DataFrame(raw_data_train,columns = ["content"])
df_test = pd.DataFrame(raw_data_test,columns = ["content"])
df_convert_train = pd.DataFrame(df_train["content"].apply(cc.convert),columns = ["content"])
df_convert_test = pd.DataFrame(df_test["content"].apply(cc.convert),columns = ["content"])
train = np.array(df_convert_train["content"].apply(data))
test = np.array(df_convert_test["content"].apply(data))
total_df_train = pd.DataFrame(train,columns = ["content"])
total_df_test = pd.DataFrame(test,columns = ["content"])

In [58]:
# SEQ_LEN: 取原文的多少個字, 建議至少256以上, 512是最大值, 但是會導致模型太大而RAM不夠用(超過的"後文"會被刪去)
SEQ_LEN = 512

In [59]:
#準備內文(x)輸入step1:此函數對文章進行分詞,並得到bert的二項輸入(準備x)
def tokenize_data(x):
  # cut with jieba 且移除標點符號
  words = "".join(jieba.cut(x))
  ids, segments = tokenizer.encode(words, max_len=SEQ_LEN)
  ids = np.array(ids)
  segments = np.array(segments, dtype ='f2')
  return ids, segments

In [60]:
#準備內文(x)輸入step2:此函數彙整每一篇文章的二項輸入,將兩種輸入分開成二個list
def prepare_input(x):
  x_tokenize = np.array(x.apply(tokenize_data)) 
  en_r = []
  seq_r = []
  for n in range(x_tokenize.shape[0]):
    en_r.append(x_tokenize[n][0])
    seq_r.append(x_tokenize[n][1])
  train_list_x = [np.array(en_r), np.array(seq_r)]
  return train_list_x

In [61]:
#準備標籤(y)輸入step1,此函數可以標記目標詞在段落中的位置
def word_position(content,keyword=keyword_list):
  keyword_set = set(keyword)
  content_word_segment= []
  content = jieba.lcut(content)
  words = []
  for word in content:
        if word not in punct:          
            words.append(word)
  content = words
  for n in content:
    test, se = tokenizer.encode(n)
    len_of_token = (len(test)-2)
    if len_of_token>0:
      if n in keyword_set:
        content_word_segment.append(100+len_of_token)
      else:
        try:
          x = n.lower()
          if x in keyword_set:
            content_word_segment.append(100+len_of_token)
          else:
            content_word_segment.append(len_of_token)
        except:
          content_word_segment.append(len_of_token)
  return np.array(content_word_segment)

In [62]:
#準備標籤(y)輸入step2,輸出元素為0和1的list,1代表該位置的字為關鍵詞的一部分
def prepare_label(content_word_segment:np.array, sq_le = SEQ_LEN):
  content_char_segment= np.array([1])
  for n in content_word_segment:
    if n >100:
      content_char_segment = np.concatenate([content_char_segment,np.ones(n-100)])
    else:
      content_char_segment = np.concatenate([content_char_segment,np.zeros(n)])
  # content_char_segment = np.concatenate([content_char_segment,np.zeros(1)])
  content_char_segment = np.concatenate([content_char_segment,np.zeros(SEQ_LEN)])
  content_char_segment = content_char_segment[:SEQ_LEN]
  return content_char_segment

In [63]:
#準備內文輸入(step1,step2)
train_x = total_df_train["content"]
train_list_x = prepare_input(train_x)

test_x = total_df_test["content"]
test_list_x = prepare_input(test_x)

#準備標籤輸入(step1,step2)
df_y_train = total_df_train["content"].apply(word_position)
train_y = df_y_train.apply(prepare_label)

df_y_test = total_df_test["content"].apply(word_position)
test_y = df_y_test.apply(prepare_label)

In [64]:
#調整標籤的資料型別
train_y = np.array(train_y.tolist())
train_y = pd.DataFrame(train_y)
train_y = train_y.astype('float32')

test_y = np.array(test_y.tolist())
test_y = pd.DataFrame(test_y)
test_y = test_y.astype('float32')

In [65]:
#讀取並微調BERT模型
import tensorflow.keras
from keras_bert import load_trained_model_from_checkpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional,LSTM

#1.讀取模型
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=False,
    trainable=False,
    seq_len=SEQ_LEN,
    )

#2.微調模型
inputs = model.inputs[:2]
x = model.output
bi_lstm = Bidirectional(LSTM(64, return_sequences=True))(x)
outputs = Dense(units=2, activation='softmax')(bi_lstm)
model = tensorflow.keras.models.Model(inputs, outputs)

#3.設置訓練方式
Optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Optimizer,
    metrics=['sparse_categorical_accuracy']
)
# model.summary()

In [66]:
# BATCH_SIZE: 每次多少個BATCH再做調整, 如果RAM不夠用就調小
EPOCHS = 10
BATCH_SIZE = 16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model.fit(
    train_list_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    callbacks=[
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=3, restore_best_weights=True)],
)

Epoch 1/10
258/258 [==============================] - 255s 890ms/step - loss: 0.4537 - sparse_categorical_accuracy: 0.7687 - val_loss: 0.3556 - val_sparse_categorical_accuracy: 0.8589
Epoch 2/10
258/258 [==============================] - 230s 893ms/step - loss: 0.3689 - sparse_categorical_accuracy: 0.8516 - val_loss: 0.3509 - val_sparse_categorical_accuracy: 0.8590
Epoch 3/10
258/258 [==============================] - 230s 893ms/step - loss: 0.3673 - sparse_categorical_accuracy: 0.8508 - val_loss: 0.3479 - val_sparse_categorical_accuracy: 0.8592
Epoch 4/10
258/258 [==============================] - 230s 892ms/step - loss: 0.3639 - sparse_categorical_accuracy: 0.8501 - val_loss: 0.3456 - val_sparse_categorical_accuracy: 0.8593
Epoch 5/10
258/258 [==============================] - 230s 893ms/step - loss: 0.3611 - sparse_categorical_accuracy: 0.8516 - val_loss: 0.3440 - val_sparse_categorical_accuracy: 0.8594
Epoch 6/10
258/258 [==============================] - 231s 894ms/step - loss: 0.

In [67]:
#此函數可以比較BERT預測的重要度序列和對應的預測文章,生成結果
def retrieve_keyword(content,content_predict_weight):
  predict_keyword_value_dic = {}
  len_of_token_list = []
  index = 1

  content = jieba.lcut(content)
  words = []
  for word in content:
        if word not in punct:          
            words.append(word)
  content = words
  for n in content:
    test, se = tokenizer.encode(n)
    len_of_token_list.append(len(test)-2)
    # print(n,len(test)-2)
    predict_keyword_value = 0
    for m in range(len(test)-2):
        if index<SEQ_LEN:
          predict_keyword_value += content_predict_weight[index]
          index += 1
          predict_keyword_value_dic[n] = predict_keyword_value/(len(test)-2)
  return predict_keyword_value_dic

In [68]:
#預測測試集(較新文章)的結果,並轉為重要度序列predict_list(0到1之間的值)
px = model.predict(test_list_x)

predict_list = []
for n in px:
  predict_list_n = []
  for m in n:
   predict_list_n.append(m[1]/(m[0]+m[1]))
  predict_list.append(predict_list_n)

In [70]:
#設定要濾除的關鍵詞
!gdown --id "1VloMIYrUdlKbPutAkmFMgQT-1vc3N9-J" --output stop_words.txt
stopword_set = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？&｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')
with open('stop_words.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = cc.convert(data)
        data = data.strip("\n")
        stopword_set.add(data)

Downloading...
From: https://drive.google.com/uc?id=1VloMIYrUdlKbPutAkmFMgQT-1vc3N9-J
To: /content/stop_words.txt
100% 2.00k/2.00k [00:00<00:00, 1.72MB/s]


In [71]:
#此函數可以彙整不同文章的關鍵詞
from collections import Counter
def merge_dict(x,y):
  X,Y=Counter(x),Counter(y)
  z=dict(X+Y)
  return(z)

In [72]:
#由重要度序列predict_list得出所有測試集上的關鍵詞重要度,並作排序
def sort_key(x):
  return x[1]

keyword_dic = {}
for n in range(len(predict_list)):
  pn_dic =retrieve_keyword(test_x.iloc[n],predict_list[n])
  pn_list = sorted(pn_dic.items(), key=lambda x:x[1], reverse=True)
  pn_list_new = []
  for m in pn_list:
    append = True
    if len(m[0]) >= 4 :

      if m[0] in stopword_set and m[0] in keyword_list_convert:
        append = False
      elif m[0] not in keyword_list_convert:
        for n in m[0]:
          if n in stopword_set:
            append = False
            if append == True:
              new_content = jieba.lcut(m[0])
              for i in new_content:
                if i in stopword_set:
                  append = False
                  break
      else: append = False
    else:
      append = False
    if append == True:
      pn_list_new.append(m)
  keyword_dic = merge_dict(keyword_dic,dict(pn_list_new))
sorted(keyword_dic.items(), key=lambda x:x[1],reverse=True)

[('https', 32.382284834980965),
 ('function', 19.994886096566916),
 ('Python', 17.754485685378317),
 ('JavaScript', 12.493102385352056),
 ('Swift', 10.708485367397467),
 ('data', 10.133803837001324),
 ('Xcode', 9.815260339528322),
 ('SwiftUI', 9.396617509424686),
 ('Google', 9.341191835701466),
 ('with', 9.214062742888927),
 ('作业系统', 8.937588961794972),
 ('view', 8.288593627512455),
 ('code', 7.91914027184248),
 ('github', 7.853953828414282),
 ('python', 6.923955190926791),
 ('index', 6.4503128826618195),
 ('Linux', 6.394973558684189),
 ('应用程式', 6.318436151370406),
 ('from', 6.277809359133244),
 ('this', 6.05574432015419),
 ('HTML', 5.980551183223724),
 ('class', 5.930173501372337),
 ('Windows', 5.893251895904541),
 ('value', 5.825365278869867),
 ('build', 5.751131504774094),
 ('image', 5.390427395701408),
 ('google', 5.363045632839203),
 ('server', 5.342373758554459),
 ('参考资料', 5.249716954305768),
 ('file', 5.204289801418781),
 ('html', 5.160533357411623),
 ('Node', 5.15554404258728),